# 🚀 Machine Learning Workflow - Modelos Avanzados (LSTM, TCN, etc.)

Notebook para implementar y evaluar modelos avanzados de series temporales para predicción de cargas en palas de turbinas eólicas.

## 📋 Objetivo

Desarrollar modelos de **Deep Learning y técnicas avanzadas** para mejorar:
- **Reducir Phase Lag** (< 200ms objetivo)
- **Mejorar NSE** (> 0.85 objetivo)
- **Reducir Peak Error** (crítico para cargas)
- **Mejor captura de dinámica temporal**

**Targets**: 
- Blade root 1 My
- Blade root 2 My

---

## 🗺️ **PLAN DE IMPLEMENTACIÓN**

### **FASE 1: FEATURES MEJORADAS** ⚡ (Rápido, alto impacto)
1. ✅ **STEP 1**: Configuración inicial + imports
2. ✅ **STEP 2**: Cargar datos existentes
3. 🆕 **STEP 3**: Crear features temporales mejoradas:
   - Lags VLOS reducidos: [0.5, 1, 2, 3, 5] segundos (vs [5-25]s actual)
   - Derivadas: d(VLOS)/dt (velocidad de cambio)
   - Aceleraciones: d²(VLOS)/dt²
   - Rolling statistics: std, mean ventanas temporales
4. 🆕 **STEP 4**: Baseline - Re-entrenar XGBoost con features mejoradas
   - Objetivo: Validar si features mejoran performance

---

### **FASE 2: LSTM/GRU** 🧠 (Máximo impacto esperado)
5. 🆕 **STEP 5**: Preparar datos en formato secuencial (ventanas temporales)
   - Window size: 50 puntos (5 segundos @ 10 Hz)
   - Sliding window approach
6. 🆕 **STEP 6**: Implementar arquitectura LSTM simple:
   ```
   Input(window_size, n_features)
   → LSTM(128 units, return_sequences=True)
   → Dropout(0.2)
   → LSTM(64 units)
   → Dropout(0.2)
   → Dense(2)  # 2 outputs (ambas palas)
   ```
7. 🆕 **STEP 7**: Entrenar LSTM con callbacks:
   - EarlyStopping
   - ModelCheckpoint
   - ReduceLROnPlateau
8. 🆕 **STEP 8**: Evaluar LSTM vs baseline

---

### **FASE 3: TEMPORAL CONVOLUTIONAL NETWORK (TCN)** ⚡ (Baja latencia)
9. 🆕 **STEP 9**: Implementar TCN:
   - Convoluciones causales (no mira al futuro)
   - Dilated convolutions para receptive field grande
   - Residual connections
10. 🆕 **STEP 10**: Entrenar y evaluar TCN

---

### **FASE 4: GRU (Variante LSTM más rápida)** 🏃
11. 🆕 **STEP 11**: Implementar GRU (similar LSTM pero más eficiente)
12. 🆕 **STEP 12**: Comparación LSTM vs GRU

---

### **FASE 5: ENSEMBLE & HÍBRIDOS** 🎭
13. 🆕 **STEP 13**: Ensemble weighted:
    - Combinar mejores modelos con pesos optimizados
14. 🆕 **STEP 14**: Kalman Filter post-procesamiento:
    - Filtrar predicciones con Kalman para reducir ruido

---

### **FASE 6: EVALUACIÓN COMPLETA** 📊
15. 🆕 **STEP 15**: Comparación exhaustiva:
    - R², RMSE, MAE, NSE, MAPE
    - Phase lag analysis
    - FFT comparison
    - Peak error
16. 🆕 **STEP 16**: Ranking y recomendación final

---

## 📁 Estructura de Carpetas

```
notebook/
├─ ML_workflow.ipynb              # Original (modelos tradicionales)
├─ ML_workflow_newModels.ipynb    # Este notebook (modelos avanzados)
├─ 01_Models_scaler/              # Scalers (reusamos)
├─ 02_Models_newGeneration/       # NUEVOS MODELOS
│  ├─ lstm_v1/
│  ├─ gru_v1/
│  ├─ tcn_v1/
│  ├─ ensemble/
│  └─ xgboost_improved/
└─ 03_Results_newModels/          # Resultados comparativos
```

---

## 🎯 Objetivos Cuantitativos

| Métrica | Baseline Actual | Objetivo LSTM | Objetivo TCN |
|---------|-----------------|---------------|-------------|
| **NSE** | 0.68-0.72 | > 0.85 | > 0.83 |
| **Phase Lag** | 700-800ms | < 200ms | < 100ms |
| **Peak Error** | 2.5M kNm | < 500k kNm | < 500k kNm |
| **MAE** | 730-830k kNm | < 400k kNm | < 400k kNm |

---

## 🔧 STEP 1: Configuración Inicial del Entorno

### 📦 Librerías:

1. **Librerías estándar**: os, sys, pathlib
2. **Análisis de datos**: pandas, numpy
3. **Visualización**: matplotlib, seaborn
4. **ML Tradicional**: scikit-learn (baseline)
5. **Deep Learning**: TensorFlow/Keras (LSTM, GRU, TCN)
6. **Utilidades**: joblib, tqdm

In [2]:
# ============================================================================
# PASO 1.1: Imports estándar
# ============================================================================

import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías estándar importadas")

# ============================================================================
# PASO 1.2: Análisis de datos
# ============================================================================

import pandas as pd
import numpy as np
import joblib

print("✅ Librerías de análisis de datos importadas")

# ============================================================================
# PASO 1.3: Visualización
# ============================================================================

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Librerías de visualización importadas")

# ============================================================================
# PASO 1.4: Machine Learning tradicional (para baseline)
# ============================================================================

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

try:
    import xgboost as xgb
    print("✅ XGBoost importado")
except ImportError:
    print("⚠️ XGBoost no disponible")
    xgb = None

# ============================================================================
# PASO 1.5: Deep Learning (TensorFlow/Keras)
# ============================================================================

try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, models, callbacks
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import (
        LSTM, GRU, Dense, Dropout, Input, 
        Conv1D, MaxPooling1D, Flatten,
        BatchNormalization, Activation
    )
    
    # Configurar para usar GPU si está disponible
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"✅ TensorFlow importado - GPU disponible: {len(gpus)} GPU(s)")
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        print("✅ TensorFlow importado - Usando CPU")
    
    print(f"   Versión TensorFlow: {tf.__version__}")
    
except ImportError:
    print("❌ TensorFlow no disponible - Instalar con: pip install tensorflow")
    tf = None

# ============================================================================
# PASO 1.6: Utilidades
# ============================================================================

try:
    from tqdm import tqdm
    print("✅ tqdm importado (progress bars)")
except ImportError:
    print("⚠️ tqdm no disponible")
    tqdm = lambda x: x  # Dummy function

# ============================================================================
# PASO 1.7: Configuración pandas
# ============================================================================

pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

print("\n" + "="*80)
print("🎉 CONFIGURACIÓN COMPLETADA")
print("="*80)

✅ Librerías estándar importadas
✅ Librerías de análisis de datos importadas
✅ Librerías de visualización importadas
✅ XGBoost importado
✅ TensorFlow importado - Usando CPU
   Versión TensorFlow: 2.20.0
✅ tqdm importado (progress bars)

🎉 CONFIGURACIÓN COMPLETADA


## 📂 STEP 2: Importar y Procesar Datos desde Simulaciones Bladed

Este paso replica exactamente la funcionalidad de `sim2csv.py` para:
1. **Leer simulaciones Bladed** (archivos `.$TE`)
2. **Extraer señales** configurables (VLOS, Blade My, Rotor speed, etc.)
3. **Feature Engineering**:
   - Crear lags de VLOS (5-25 segundos)
   - Crear componentes sin/cos del azimuth
4. **Exportar a CSV** con todas las features

### 📋 Configuración:
- **Input**: Simulaciones Bladed en `U:\Studies\437_lidar_VLOS_IPC\Outputs\OPT3\3600s`
- **Output**: CSVs en carpeta del notebook con features engineered
- **Features**: 181 features (VLOS lags + sin/cos + targets)

### 2.1: Configuración de Paths y Variables

**Modificar estas variables según tus necesidades:**

In [ ]:
# ============================================================================
# CONFIGURACIÓN - MODIFICAR SEGÚN TUS NECESIDADES
# ============================================================================

# 1. Ruta de postprocessbladed
POSTPROCESSBLADED_PATH = r"C:\Users\aitorredondoruiz\Desktop\2B_energy\__Git"

# 2. Ruta donde están las simulaciones de Bladed (archivos .$TE)
LOADPATH = r"U:\Studies\437_lidar_VLOS_IPC\Outputs\OPT3\3600s"

# 3. Lista de nombres de archivos de simulación a procesar
FILE_NAMES =[
    "0001_DLC1.2_030_000_000_01",
    "0002_DLC1.2_030_000_000_02",
    "0003_DLC1.2_030_000_000_03",
    "0004_DLC1.2_030_000_000_04",
    "0005_DLC1.2_030_000_000_05",
    "0006_DLC1.2_030_000_000_06",
    "0007_DLC1.2_050_000_000_01",
    "0008_DLC1.2_050_000_000_02",
    "0009_DLC1.2_050_000_000_03",
    "0010_DLC1.2_050_000_000_04",
    "0011_DLC1.2_050_000_000_05",
    "0012_DLC1.2_050_000_000_06",
    "0013_DLC1.2_070_000_000_01",
    "0014_DLC1.2_070_000_000_02",
    "0015_DLC1.2_070_000_000_03",
    "0016_DLC1.2_070_000_000_04",
    "0017_DLC1.2_070_000_000_05",
    "0018_DLC1.2_070_000_000_06",
    "0019_DLC1.2_090_000_000_01",
    "0020_DLC1.2_090_000_000_02",
    "0021_DLC1.2_090_000_000_03",
    "0022_DLC1.2_090_000_000_04",
    "0023_DLC1.2_090_000_000_05",
    "0024_DLC1.2_090_000_000_06",
    "0025_DLC1.2_110_000_000_01",
    "0026_DLC1.2_110_000_000_02",
    "0027_DLC1.2_110_000_000_03",
    "0028_DLC1.2_110_000_000_04",
    "0029_DLC1.2_110_000_000_05",
    "0030_DLC1.2_110_000_000_06",
    "0031_DLC1.2_130_000_000_01",
    "0032_DLC1.2_130_000_000_02",
    "0033_DLC1.2_130_000_000_03",
    "0034_DLC1.2_130_000_000_04",
    "0035_DLC1.2_130_000_000_05",
    "0036_DLC1.2_130_000_000_06",
    "0037_DLC1.2_150_000_000_01",
    "0038_DLC1.2_150_000_000_02",
    "0039_DLC1.2_150_000_000_03",
    "0040_DLC1.2_150_000_000_04",
    "0041_DLC1.2_150_000_000_05",
    "0042_DLC1.2_150_000_000_06",
    "0043_DLC1.2_170_000_000_01",
    "0044_DLC1.2_170_000_000_02",
    "0045_DLC1.2_170_000_000_03",
    "0046_DLC1.2_170_000_000_04",
    "0047_DLC1.2_170_000_000_05",
    "0048_DLC1.2_170_000_000_06",
    "0049_DLC1.2_190_000_000_01",
    "0050_DLC1.2_190_000_000_02",
    "0051_DLC1.2_190_000_000_03",
    "0052_DLC1.2_190_000_000_04",
    "0053_DLC1.2_190_000_000_05",
    "0054_DLC1.2_190_000_000_06",
    "0055_DLC1.2_210_000_000_01",
    "0056_DLC1.2_210_000_000_02",
    "0057_DLC1.2_210_000_000_03",
    "0058_DLC1.2_210_000_000_04",
    "0059_DLC1.2_210_000_000_05",
    "0060_DLC1.2_210_000_000_06",
    "0061_DLC1.2_230_000_000_01",
    "0062_DLC1.2_230_000_000_02",
    "0063_DLC1.2_230_000_000_03",
    "0064_DLC1.2_230_000_000_04",
    "0065_DLC1.2_230_000_000_05",
    "0066_DLC1.2_230_000_000_06",
    "0067_DLC1.2_250_000_000_01",
    "0068_DLC1.2_250_000_000_02",
    "0069_DLC1.2_250_000_000_03",
    "0070_DLC1.2_250_000_000_04",
    "0071_DLC1.2_250_000_000_05",
    "0072_DLC1.2_250_000_000_06",
]
                                            

# 4. Ruta donde guardar los CSV resultantes (carpeta actual del notebook)
RESULTSPATH = r"C:\Users\aitorredondoruiz\Desktop\2B_energy\__Git\Lidar_My_validation_VLOS\data_train_NEW_ML"

# 5. Opciones de procesamiento
ADD_UNITS = False  # Añadir fila de unidades en CSV

# 6. Feature Engineering - Lags de VLOS
CREATE_LAGS = False  # Crear lags de VLOS
LAG_SECONDS = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

# 7. Feature Engineering - Componentes trigonométricas
CREATE_AZIMUTH_COMPONENTS = True  # Crear sin/cos del azimuth

# 8. Posiciones radiales para señales Aero (si se usan)
AERO_POSITIONS = [0.0, 6.0, 18.0, 30.0, 46.0, 59.0, 68.25]

# 9. Diccionarios de variables a extraer (organizadas por señal de Bladed)
VAR_DICTS = {
    "Hub_rotating": {
        "Hub_rotating": [
            "Blade root 1 My",  # Momento flector pala 1 (TARGET)
            "Blade root 2 My"   # Momento flector pala 2 (TARGET)
        ]
    },
    "Pitch_actuator": {
        "Pitch_actuator": [
            "Blade 1 pitch angle",  # Ángulo de pala 1
            "Blade 2 pitch angle",  # Ángulo de pala 2
        ]
    },
    "Drive_train": {
        "Drive_train": ["Rotor azimuth angle"]  # Ángulo azimutal del rotor
    },
    "Summary": {
        "Summary": ["Rotor speed"]  # Velocidad del rotor
    },
    "External_controller": {
        "External_controller": [
            "LAC_VLOS_BEAM0_RANGE5",
            "LAC_VLOS_BEAM1_RANGE5",
            "LAC_VLOS_BEAM2_RANGE5",
            "LAC_VLOS_BEAM3_RANGE5",
            "LAC_VLOS_BEAM4_RANGE5",
            "LAC_VLOS_BEAM5_RANGE5",
            "LAC_VLOS_BEAM6_RANGE5",
            "LAC_VLOS_BEAM7_RANGE5",
        ]
    }
}

print("=" * 80)
print("⚙️  CONFIGURACIÓN CARGADA")
print("=" * 80)
print(f"\n📁 Path postprocessbladed: {POSTPROCESSBLADED_PATH}")
print(f"📁 Path simulaciones: {LOADPATH}")
print(f"📁 Path resultados: {RESULTSPATH}")
print(f"\n📊 Archivos a procesar: {len(FILE_NAMES)}")
for fname in FILE_NAMES:
    print(f"   • {fname}")
print(f"\n🔧 Feature Engineering:")
print(f"   • Lags VLOS: {CREATE_LAGS} ({len(LAG_SECONDS)} lags: {LAG_SECONDS[0]}-{LAG_SECONDS[-1]}s)")
print(f"   • Sin/Cos Azimuth: {CREATE_AZIMUTH_COMPONENTS}")
print(f"\n✅ Configuración lista")

### 2.2: Importar postprocessbladed y Definir Funciones Auxiliares

In [ ]:
# ============================================================================
# Importar postprocessbladed
# ============================================================================

# Añadir path al sys.path
if POSTPROCESSBLADED_PATH not in sys.path:
    sys.path.insert(0, POSTPROCESSBLADED_PATH)

# Importar postprocessbladed (esto también importa numpy y pandas)
import postprocessbladed.postprocessbladed as pp

print("✅ postprocessbladed importado correctamente")
print(f"   Versión numpy: {np.__version__}")
print(f"   Versión pandas: {pd.__version__}")

### 2.3: Funciones de Feature Engineering

In [ ]:
def create_vlos_lags(df, lag_seconds_list):
    """
    Crea features de lag para las variables de velocidad del viento (VLOS).
    
    Args:
        df: DataFrame con los datos
        lag_seconds_list: Lista de lags en segundos a crear
    
    Returns:
        DataFrame con las nuevas columnas de lag añadidas
    """
    print(f"\n[Feature Engineering] Creando lags de VLOS...")
    
    # Identificar columnas de velocidad del viento (VLOS)
    vlos_columns = [col for col in df.columns if 'LAC_VLOS' in col and '_lag' not in col]
    
    print(f"   Variables VLOS encontradas: {len(vlos_columns)}")
    
    # Calcular tiempo de muestreo (dt)
    if 'Time' in df.columns and len(df) > 1:
        dt = df['Time'].iloc[1] - df['Time'].iloc[0]
        print(f"   Tiempo de muestreo: {dt:.4f} segundos")
    else:
        dt = 0.05  # Default 20Hz
        print(f"   Tiempo de muestreo por defecto: {dt} segundos")
    
    # Crear lags para cada variable VLOS
    print(f"   Creando {len(lag_seconds_list)} lags para {len(vlos_columns)} variables...")
    
    total_created = 0
    for vlos_col in vlos_columns:
        for lag_sec in lag_seconds_list:
            # Calcular número de muestras para el lag
            lag_samples = int(round(lag_sec / dt))
            
            # Crear nombre de la nueva columna
            new_col_name = f"{vlos_col}_lag{lag_sec}s"
            
            # Crear la columna con shift
            df[new_col_name] = df[vlos_col].shift(lag_samples)
            
            total_created += 1
    
    print(f"   ✅ {total_created} features de lag creadas")
    return df


def create_azimuth_components(df):
    """
    Crea componentes seno y coseno del ángulo de azimuth del rotor.
    
    Args:
        df: DataFrame con los datos
    
    Returns:
        DataFrame con las nuevas columnas sin/cos añadidas
    """
    print(f"\n[Feature Engineering] Creando componentes trigonométricas del azimuth...")
    
    azimuth_col = 'Rotor azimuth angle'
    
    if azimuth_col not in df.columns:
        print(f"   ⚠️ Columna '{azimuth_col}' no encontrada, saltando...")
        return df
    
    # Verificar rango de valores para determinar unidades
    max_val = df[azimuth_col].max()
    
    if max_val > 6.5:  # Si es > 2*pi, probablemente en grados
        print(f"   Rango detectado: 0-{max_val:.1f}° (grados)")
        # Convertir de grados a radianes
        azimuth_rad = np.deg2rad(df[azimuth_col])
    else:
        print(f"   Rango detectado: 0-{max_val:.1f} (radianes)")
        azimuth_rad = df[azimuth_col]
    
    # Crear componentes
    df['sin_rotor_azimuth'] = np.sin(azimuth_rad)
    df['cos_rotor_azimuth'] = np.cos(azimuth_rad)
    
    print(f"   ✅ Creadas: sin_rotor_azimuth, cos_rotor_azimuth")
    return df


def remove_nan_rows(df):
    """
    Elimina las filas que contienen valores NaN.
    Esto ocurre al inicio debido a los lags.
    
    Args:
        df: DataFrame con los datos
    
    Returns:
        DataFrame sin filas con NaN
    """
    print(f"\n[Limpieza] Eliminando filas con NaN...")
    
    original_rows = len(df)
    df_clean = df.dropna()
    removed_rows = original_rows - len(df_clean)
    
    print(f"   Filas originales: {original_rows:,}")
    print(f"   Filas eliminadas: {removed_rows:,}")
    print(f"   Filas finales: {len(df_clean):,}")
    
    return df_clean


print("✅ Funciones de feature engineering definidas")

### 2.4: Procesar Simulaciones y Generar CSVs

**Este proceso:**
1. Lee archivos binarios Bladed (.$TE)
2. Extrae señales configuradas
3. Aplica feature engineering
4. Guarda CSV con todas las features

In [ ]:
# ============================================================================
# PROCESAMIENTO PRINCIPAL
# ============================================================================

print("\n" + "=" * 80)
print("🚀 INICIO DEL PROCESAMIENTO")
print("=" * 80)

# Procesar cada archivo de simulación
for file_name in FILE_NAMES:
    print(f"\n{'='*80}")
    print(f"📄 Procesando: {file_name}")
    print("=" * 80)
    
    # 1. Leer señales de Bladed
    print("\n[1/5] Leyendo señales de Bladed...")
    
    # Construir path completo
    full_path = os.path.join(LOADPATH, file_name)
    
    # Extraer señales usando postprocessbladed
    timeseries = pp.extract_csv_from_binary(
        [full_path],
        VAR_DICTS,
        add_units=ADD_UNITS
    )
    
    # 2. Convertir a DataFrame
    print("\n[2/5] Convirtiendo a DataFrame...")
    df = pd.DataFrame(timeseries)
    print(f"   Shape inicial: {df.shape}")
    print(f"   Columnas: {list(df.columns)}")
    
    # 3. Feature Engineering - Lags
    if CREATE_LAGS:
        df = create_vlos_lags(df, LAG_SECONDS)
        print(f"   Shape después de lags: {df.shape}")
    
    # 4. Feature Engineering - Azimuth
    if CREATE_AZIMUTH_COMPONENTS:
        df = create_azimuth_components(df)
        print(f"   Shape después de azimuth: {df.shape}")
    
    # 5. Eliminar NaN
    print("\n[3/5] Limpiando datos...")
    df_clean = remove_nan_rows(df)
    
    # 6. Guardar CSV
    print("\n[4/5] Guardando CSV...")
    output_filename = f"{file_name}.csv"
    output_path = RESULTSPATH / output_filename
    df_clean.to_csv(output_path, index=False)
    
    print(f"   ✅ CSV guardado: {output_path}")
    print(f"   ✅ Shape final: {df_clean.shape}")
    print(f"   ✅ Columnas totales: {len(df_clean.columns)}")
    
    # 7. Resumen
    print("\n[5/5] Resumen:")
    print(f"   • Filas: {len(df_clean):,}")
    print(f"   • Columnas: {len(df_clean.columns)}")
    print(f"   • Memoria: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   • Features creadas:")
    if CREATE_LAGS:
        lag_cols = [c for c in df_clean.columns if '_lag' in c]
        print(f"      - Lags VLOS: {len(lag_cols)}")
    if CREATE_AZIMUTH_COMPONENTS:
        print(f"      - Sin/Cos azimuth: 2")
    
    print(f"\n✅ Archivo {file_name} procesado exitosamente!")

print("\n" + "=" * 80)
print("🎉 PROCESAMIENTO COMPLETADO")
print("=" * 80)
print(f"\n📊 Total archivos procesados: {len(FILE_NAMES)}")
print(f"📁 Ubicación: {RESULTSPATH}")